In [ ]:
!pip install plotly
!pip install pandas
!pip install datetime
!pip install numpy
!pip install python-binance
!pip install ipywidgets
!pip install ta

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user insta

# Mettre à jour les data

In [34]:
import os
import time
import pandas as pd
from datetime import datetime
from binance.client import Client

# Connexion à l'API Binance 
client = Client()

# Répertoire pour les fichiers CSV
save_dir = "Historique"
os.makedirs(save_dir, exist_ok=True)

# Paires et timeframes à gérer
pairs = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "DOGEUSDT"]
timeframes = {
    "1h": Client.KLINE_INTERVAL_1HOUR,
    "4h": Client.KLINE_INTERVAL_4HOUR,
    "1d": Client.KLINE_INTERVAL_1DAY
}
start_date_str = "1 Jan, 2017"

# Fonction pour télécharger tout l'historique d'une paire / interval
def get_full_history(symbol, interval, start_str):
    print(f"⬇️ Téléchargement : {symbol} - {interval}")
    all_data = []
    start_time = int(datetime.strptime(start_str, "%d %b, %Y").timestamp() * 1000)

    while True:
        try:
            data = client.get_klines(
                symbol=symbol,
                interval=interval,
                startTime=start_time,
                limit=1000
            )
            if not data:
                break

            df = pd.DataFrame(data, columns=[
                "open_time", "open", "high", "low", "close", "volume",
                "close_time", "quote_asset_volume", "number_of_trades",
                "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
            ])
            df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
            df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')

            all_data.append(df)

            start_time = int(data[-1][6]) + 1  # close_time + 1ms
            time.sleep(1)
        except Exception as e:
            print(f"❌ Erreur : {e}")
            break

    return pd.concat(all_data) if all_data else pd.DataFrame()

# Fonction de mise à jour d'un fichier CSV
def update_history(symbol, interval_key):
    interval = timeframes[interval_key]
    filename = f"{symbol}_{interval_key}.csv"
    filepath = os.path.join(save_dir, filename)

    if os.path.exists(filepath):
        print(f"🔁 Mise à jour : {filename}")
        df_existing = pd.read_csv(filepath)
        df_existing["close_time"] = pd.to_datetime(df_existing["close_time"])

        last_timestamp = int(df_existing["close_time"].max().timestamp() * 1000) + 1
        new_data = []

        while True:
            try:
                data = client.get_klines(
                    symbol=symbol,
                    interval=interval,
                    startTime=last_timestamp,
                    limit=1000
                )
                if not data:
                    break

                df = pd.DataFrame(data, columns=[
                    "open_time", "open", "high", "low", "close", "volume",
                    "close_time", "quote_asset_volume", "number_of_trades",
                    "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
                ])
                df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
                df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')

                new_data.append(df)

                last_timestamp = int(data[-1][6]) + 1
                time.sleep(1)
            except Exception as e:
                print(f"❌ Erreur MAJ : {e}")
                break

        if new_data:
            df_new = pd.concat(new_data)
            df_combined = pd.concat([df_existing, df_new])
            df_combined.drop_duplicates(subset="close_time", inplace=True)
            df_combined.to_csv(filepath, index=False)
            print(f"✅ Fichier mis à jour : {filename} ({len(df_combined)} lignes)")
        else:
            print("✅ Aucune nouvelle donnée.")
    else:
        print(f"📁 Création : {filename}")
        df = get_full_history(symbol, interval, start_date_str)
        if not df.empty:
            df.to_csv(filepath, index=False)
            print(f"✅ Fichier créé : {filename} ({len(df)} lignes)")
        else:
            print("❌ Aucune donnée trouvée.")

# Lancer le script pour toutes les paires et timeframes
for pair in pairs:
    for tf in timeframes.keys():
        update_history(pair, tf)


🔁 Mise à jour : BTCUSDT_1h.csv
✅ Fichier mis à jour : BTCUSDT_1h.csv (67557 lignes)
🔁 Mise à jour : BTCUSDT_4h.csv
✅ Fichier mis à jour : BTCUSDT_4h.csv (16906 lignes)
🔁 Mise à jour : BTCUSDT_1d.csv
✅ Fichier mis à jour : BTCUSDT_1d.csv (2821 lignes)
🔁 Mise à jour : ETHUSDT_1h.csv
✅ Fichier mis à jour : ETHUSDT_1h.csv (67557 lignes)
🔁 Mise à jour : ETHUSDT_4h.csv
✅ Fichier mis à jour : ETHUSDT_4h.csv (16906 lignes)
🔁 Mise à jour : ETHUSDT_1d.csv
✅ Fichier mis à jour : ETHUSDT_1d.csv (2821 lignes)
🔁 Mise à jour : BNBUSDT_1h.csv
✅ Fichier mis à jour : BNBUSDT_1h.csv (65620 lignes)
🔁 Mise à jour : BNBUSDT_4h.csv
✅ Fichier mis à jour : BNBUSDT_4h.csv (16421 lignes)
🔁 Mise à jour : BNBUSDT_1d.csv
✅ Fichier mis à jour : BNBUSDT_1d.csv (2740 lignes)
🔁 Mise à jour : SOLUSDT_1h.csv
✅ Fichier mis à jour : SOLUSDT_1h.csv (41503 lignes)
🔁 Mise à jour : SOLUSDT_4h.csv
✅ Fichier mis à jour : SOLUSDT_4h.csv (10382 lignes)
🔁 Mise à jour : SOLUSDT_1d.csv
✅ Fichier mis à jour : SOLUSDT_1d.csv (1731 lign

# Analyse 

Cette section se concentre sur l'analyse technique des données avec les paramètres sélectionnés.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# Installation des bibliothèques si nécessaire
try:
    import ta
    from scipy import stats
except ImportError:
    print("Installation des bibliothèques nécessaires...")
    !pip install ta
    !pip install plotly
    !pip install scikit-learn
    !pip install scipy
    import ta
    from scipy import stats

# Chargement des données ETH/USDT (1h)
file_path = "./Historique/ETHUSDT_1h.csv"
df = pd.read_csv(file_path)

# Préparation des données
df['open_time'] = pd.to_datetime(df['open_time'])
df['close_time'] = pd.to_datetime(df['close_time'])
df.set_index('open_time', inplace=True)
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)

# Afficher les informations de base
print(f"Données ETH/USDT 1h de {df.index.min()} à {df.index.max()}")
print(f"Nombre de périodes: {len(df)}")
print(f"Prix actuel: ${df['close'].iloc[-1]:.2f}")
print(f"Variation 24h: {((df['close'].iloc[-1]/df['close'].iloc[-24])-1)*100:.2f}%")

# Filtrer les données pour ne garder que les derniers X jours
days_to_analyze = 5  # Ajustable
cutoff_date = df.index.max() - pd.Timedelta(days=days_to_analyze)
df_recent = df[df.index > cutoff_date]

# Ajouter un index numérique pour simplifier les calculs des lignes de tendance
df_recent['candle_index'] = range(len(df_recent))

# Calcul des indicateurs techniques avec la bibliothèque 'ta' au lieu de 'talib'
# 1. Moyennes mobiles (SMA)
df_recent['SMA20'] = ta.trend.sma_indicator(df_recent['close'], window=20)
df_recent['SMA50'] = ta.trend.sma_indicator(df_recent['close'], window=50)
df_recent['SMA200'] = ta.trend.sma_indicator(df_recent['close'], window=200)

# 2. RSI (Relative Strength Index)
df_recent['RSI'] = ta.momentum.rsi(df_recent['close'], window=14)

# 3. MACD (Moving Average Convergence Divergence)
macd = ta.trend.MACD(df_recent['close'], window_slow=26, window_fast=12, window_sign=9)
df_recent['MACD'] = macd.macd()
df_recent['MACD_signal'] = macd.macd_signal()
df_recent['MACD_hist'] = macd.macd_diff()  # Histogramme = MACD - Signal

# 4. Bandes de Bollinger
bollinger = ta.volatility.BollingerBands(df_recent['close'], window=20, window_dev=2)
df_recent['upper_band'] = bollinger.bollinger_hband()
df_recent['middle_band'] = bollinger.bollinger_mavg()
df_recent['lower_band'] = bollinger.bollinger_lband()

# 5. ATR (Average True Range) pour la volatilité
df_recent['ATR'] = ta.volatility.average_true_range(df_recent['high'], df_recent['low'], df_recent['close'], window=14)

# Fonction pour identifier les niveaux de support et résistance
def find_support_resistance(df, window=5, threshold=0.02):
    highs = []
    lows = []
    
    for i in range(window, len(df) - window):
        # Check for swing high
        if all(df['high'].iloc[i] > df['high'].iloc[i-j] for j in range(1, window+1)) and \
           all(df['high'].iloc[i] > df['high'].iloc[i+j] for j in range(1, window+1)):
            highs.append((df.index[i], df['high'].iloc[i]))
            
        # Check for swing low
        if all(df['low'].iloc[i] < df['low'].iloc[i-j] for j in range(1, window+1)) and \
           all(df['low'].iloc[i] < df['low'].iloc[i+j] for j in range(1, window+1)):
            lows.append((df.index[i], df['low'].iloc[i]))
    
    # Group similar price levels
    resistance_levels = []
    support_levels = []
    
    for high_date, high_price in highs:
        # Check if similar price already exists
        exists = False
        for i, (_, r_price) in enumerate(resistance_levels):
            if abs(high_price - r_price) / r_price < threshold:
                exists = True
                # Take average of the prices
                resistance_levels[i] = (resistance_levels[i][0], (resistance_levels[i][1] + high_price) / 2)
                break
        if not exists:
            resistance_levels.append((high_date, high_price))
    
    for low_date, low_price in lows:
        # Check if similar price already exists
        exists = False
        for i, (_, s_price) in enumerate(support_levels):
            if abs(low_price - s_price) / s_price < threshold:
                exists = True
                # Take average of the prices
                support_levels[i] = (support_levels[i][0], (support_levels[i][1] + low_price) / 2)
                break
        if not exists:
            support_levels.append((low_date, low_price))
    
    return support_levels, resistance_levels

# Fonction pour détecter les lignes de tendance obliques (supports et résistances dynamiques)
def find_trend_lines(df, min_points=3, max_lines=3, price_tolerance=0.005):
    """Identifier les lignes de tendance avec au moins min_points points de contact.
    
    Args:
        df: DataFrame avec au moins les colonnes 'candle_index', 'high', 'low'
        min_points: nombre minimum de points requis pour une ligne de tendance valide
        max_lines: nombre maximum de lignes à retourner (les meilleures)
        price_tolerance: tolérance pour considérer un point comme un touch point (en %)
        
    Returns:
        tuple: (trend_supports, trend_resistances)
    """
    
    # Préparer les données pour les calculs
    highs = [(x, y) for x, y in zip(df['candle_index'], df['high'])]
    lows = [(x, y) for x, y in zip(df['candle_index'], df['low'])]
    
    # Fonction pour calculer la qualité d'une ligne
    def line_quality(line_params, points, is_support=True):
        """Évalue la qualité d'une ligne de tendance."""
        a, b = line_params  # y = ax + b
        
        touch_points = []
        errors = []
        violations = 0
        
        for x, y in points:
            line_y = a * x + b
            error = (y - line_y) / y  # erreur relative
            errors.append(abs(error))
            
            # Déterminer si c'est un point de contact avec tolérance
            if abs(error) <= price_tolerance:
                touch_points.append((x, y))
                
            # Si c'est un support, on doit avoir line_y <= y
            # Si c'est une résistance, on doit avoir line_y >= y
            if (is_support and line_y > y) or (not is_support and line_y < y):
                violations += 1
        
        # Calculer la qualité (plus de points, moins d'erreur, moins de violations = meilleur)
        if len(touch_points) < min_points or violations > 0:
            return -1, None  # Ligne invalide
        
        # Importance = nombre de points * longueur relative entre premier et dernier point
        if touch_points:
            touch_points.sort(key=lambda p: p[0])  # Trier par x
            x_span = touch_points[-1][0] - touch_points[0][0]
            importance = len(touch_points) * (x_span / len(df))
            
            # Calculer la pente en degrés
            angle_deg = abs(np.degrees(np.arctan(a)))
            # Pénaliser les lignes trop verticales (>45°) ou trop horizontales (<2°)
            if angle_deg > 45 or angle_deg < 2:
                importance *= 0.5
                
            return importance, touch_points
        return -1, None
    
    # Trouver les supports (lignes de tendance sous les prix)
    support_lines = []
    for i in range(len(lows)):
        for j in range(i + 1, len(lows)):
            x1, y1 = lows[i]
            x2, y2 = lows[j]
            if x1 == x2:  # Éviter la division par zéro
                continue
            
            # Calculer les paramètres de la ligne (y = ax + b)
            a = (y2 - y1) / (x2 - x1)
            b = y1 - a * x1
            
            # Évaluer la qualité de la ligne
            importance, touch_points = line_quality((a, b), lows, is_support=True)
            if importance > 0 and touch_points is not None:
                support_lines.append((importance, a, b, touch_points))
    
    # Trouver les résistances (lignes de tendance au-dessus des prix)
    resistance_lines = []
    for i in range(len(highs)):
        for j in range(i + 1, len(highs)):
            x1, y1 = highs[i]
            x2, y2 = highs[j]
            if x1 == x2:  # Éviter la division par zéro
                continue
            
            # Calculer les paramètres de la ligne (y = ax + b)
            a = (y2 - y1) / (x2 - x1)
            b = y1 - a * x1
            
            # Évaluer la qualité de la ligne
            importance, touch_points = line_quality((a, b), highs, is_support=False)
            if importance > 0 and touch_points is not None:
                resistance_lines.append((importance, a, b, touch_points))
    
    # Trier par importance et prendre les meilleures lignes
    support_lines.sort(key=lambda x: x[0], reverse=True)
    resistance_lines.sort(key=lambda x: x[0], reverse=True)
    
    # Supprimer les doublons (lignes très similaires)
    def filter_similar_lines(lines, tolerance=0.01):
        if not lines:
            return []
            
        filtered_lines = [lines[0]]
        for line in lines[1:]:
            _, a1, b1, _ = line
            is_duplicate = False
            
            for _, a2, b2, _ in filtered_lines:
                # Comparer les pentes et les ordonnées à l'origine
                if abs(a1 - a2) < tolerance and abs(b1 - b2) / (abs(b1) + abs(b2) + 1e-10) < tolerance:
                    is_duplicate = True
                    break
                    
            if not is_duplicate:
                filtered_lines.append(line)
                
        return filtered_lines[:max_lines]
    
    return (filter_similar_lines(support_lines), 
            filter_similar_lines(resistance_lines))

# Identifier les niveaux de support et résistance
support_levels, resistance_levels = find_support_resistance(df_recent)

# Fonction pour identifier les tendances
def identify_trend(df, window=20):
    # Utiliser la régression linéaire pour déterminer la tendance
    X = np.array(range(window)).reshape(-1, 1)
    y = df['close'].iloc[-window:].values
    model = LinearRegression().fit(X, y)
    slope = model.coef_[0]
    
    # Calculer la force de la tendance (R²)
    y_pred = model.predict(X)
    r_squared = 1 - np.sum((y - y_pred) ** 2) / np.sum((y - np.mean(y)) ** 2)
    
    if slope > 0 and r_squared > 0.6:
        return "Haussière (forte)", slope, r_squared
    elif slope > 0:
        return "Haussière (faible)", slope, r_squared
    elif slope < 0 and r_squared > 0.6:
        return "Baissière (forte)", slope, r_squared
    elif slope < 0:
        return "Baissière (faible)", slope, r_squared
    else:
        return "Neutre", slope, r_squared

# Analyser la tendance actuelle
trend, slope, r_squared = identify_trend(df_recent)

# Identifier les lignes de tendance dynamiques
trend_supports, trend_resistances = find_trend_lines(df_recent, min_points=3, max_lines=3)

print(f"\nTendance actuelle: {trend}")
print(f"Force de la tendance (R²): {r_squared:.2f}")
print(f"Lignes de support dynamique trouvées: {len(trend_supports)}")
print(f"Lignes de résistance dynamique trouvées: {len(trend_resistances)}")

# Création du dashboard interactif avec Plotly
fig = make_subplots(rows=4, cols=1, 
                     shared_xaxes=True,
                     vertical_spacing=0.02,
                     row_heights=[0.5, 0.15, 0.15, 0.2],
                     subplot_titles=("ETH/USDT - Prix et Tendance", "Volume", "RSI", "MACD"))

# Graphique des prix avec bougies japonaises
fig.add_trace(go.Candlestick(x=df_recent.index,
                            open=df_recent['open'],
                            high=df_recent['high'],
                            low=df_recent['low'],
                            close=df_recent['close'],
                            name="Prix"), row=1, col=1)

# Ajouter les moyennes mobiles
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['SMA20'], name="SMA 20", line=dict(color="blue", width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['SMA50'], name="SMA 50", line=dict(color="orange", width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['SMA200'], name="SMA 200", line=dict(color="red", width=1)), row=1, col=1)

# Ajouter les bandes de Bollinger
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['upper_band'], name="BB Upper", line=dict(color="rgba(250, 0, 0, 0.4)", width=1)), row=1, col=1)
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['lower_band'], name="BB Lower", line=dict(color="rgba(250, 0, 0, 0.4)", width=1), fill='tonexty', fillcolor='rgba(250, 0, 0, 0.1)'), row=1, col=1)

# Ajouter les niveaux de support et résistance
current_price = df_recent['close'].iloc[-1]
for _, price in resistance_levels:
    if price > current_price * 0.9 and price < current_price * 1.2:  # Montrer seulement les niveaux pertinents
        fig.add_shape(type="line", x0=df_recent.index[0], x1=df_recent.index[-1], y0=price, y1=price,
                    line=dict(color="red", width=1, dash="dash"), row=1, col=1)
        fig.add_annotation(x=df_recent.index[-1], y=price, text=f"R: {price:.2f}",
                        showarrow=False, xanchor="right", row=1, col=1)

for _, price in support_levels:
    if price > current_price * 0.8 and price < current_price * 1.1:  # Montrer seulement les niveaux pertinents
        fig.add_shape(type="line", x0=df_recent.index[0], x1=df_recent.index[-1], y0=price, y1=price,
                    line=dict(color="green", width=1, dash="dash"), row=1, col=1)
        fig.add_annotation(x=df_recent.index[-1], y=price, text=f"S: {price:.2f}",
                        showarrow=False, xanchor="right", row=1, col=1)

# Ajouter les lignes de tendance dynamiques (supports obliques)
for i, (_, a, b, touch_points) in enumerate(trend_supports):
    # Obtenir la première et dernière date pour tracer la ligne
    start_idx = min(tp[0] for tp in touch_points)
    end_idx = max(tp[0] for tp in touch_points)
    
    # Prolonger la ligne un peu au-delà des points pour la visibilité
    extended_start_idx = max(0, start_idx - 10)
    extended_end_idx = min(len(df_recent) - 1, end_idx + 20)
    
    # Tracer la ligne de tendance
    x_start = df_recent.iloc[extended_start_idx].name
    x_end = df_recent.iloc[extended_end_idx].name
    y_start = a * (extended_start_idx) + b
    y_end = a * (extended_end_idx) + b
    
    # Ajouter la ligne comme une forme
    fig.add_shape(
        type="line",
        x0=x_start, y0=y_start,
        x1=x_end, y1=y_end,
        line=dict(color="green", width=2),
        row=1, col=1
    )
    
    # Ajouter des marqueurs pour les points de contact
    for candle_idx, price in touch_points:
        date = df_recent.iloc[candle_idx].name
        fig.add_trace(
            go.Scatter(
                x=[date],
                y=[price],
                mode='markers',
                marker=dict(color='green', size=8, symbol='circle-open'),
                showlegend=False
            ),
            row=1, col=1
        )

# Ajouter les lignes de tendance dynamiques (résistances obliques)
for i, (_, a, b, touch_points) in enumerate(trend_resistances):
    # Obtenir la première et dernière date pour tracer la ligne
    start_idx = min(tp[0] for tp in touch_points)
    end_idx = max(tp[0] for tp in touch_points)
    
    # Prolonger la ligne un peu au-delà des points pour la visibilité
    extended_start_idx = max(0, start_idx - 10)
    extended_end_idx = min(len(df_recent) - 1, end_idx + 20)
    
    # Tracer la ligne de tendance
    x_start = df_recent.iloc[extended_start_idx].name
    x_end = df_recent.iloc[extended_end_idx].name
    y_start = a * (extended_start_idx) + b
    y_end = a * (extended_end_idx) + b
    
    # Ajouter la ligne comme une forme
    fig.add_shape(
        type="line",
        x0=x_start, y0=y_start,
        x1=x_end, y1=y_end,
        line=dict(color="red", width=2),
        row=1, col=1
    )
    
    # Ajouter des marqueurs pour les points de contact
    for candle_idx, price in touch_points:
        date = df_recent.iloc[candle_idx].name
        fig.add_trace(
            go.Scatter(
                x=[date],
                y=[price],
                mode='markers',
                marker=dict(color='red', size=8, symbol='circle-open'),
                showlegend=False
            ),
            row=1, col=1
        )

# Volume
colors = ['red' if row['close'] < row['open'] else 'green' for _, row in df_recent.iterrows()]
fig.add_trace(go.Bar(x=df_recent.index, y=df_recent['volume'], name="Volume", marker_color=colors), row=2, col=1)

# RSI
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['RSI'], name="RSI", line=dict(color="purple", width=1)), row=3, col=1)
# Ajouter des lignes horizontales pour les niveaux RSI
fig.add_shape(type="line", x0=df_recent.index[0], x1=df_recent.index[-1], y0=70, y1=70,
            line=dict(color="red", width=1, dash="dash"), row=3, col=1)
fig.add_shape(type="line", x0=df_recent.index[0], x1=df_recent.index[-1], y0=30, y1=30,
            line=dict(color="green", width=1, dash="dash"), row=3, col=1)

# MACD
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['MACD'], name="MACD", line=dict(color="blue", width=1)), row=4, col=1)
fig.add_trace(go.Scatter(x=df_recent.index, y=df_recent['MACD_signal'], name="Signal", line=dict(color="red", width=1)), row=4, col=1)

# Histogramme MACD
colors = ['red' if val < 0 else 'green' for val in df_recent['MACD_hist']]
fig.add_trace(go.Bar(x=df_recent.index, y=df_recent['MACD_hist'], name="Histogramme", marker_color=colors), row=4, col=1)

# Mise en forme du graphique
fig.update_layout(
    title=f"Analyse Technique ETH/USDT 1h - {datetime.now().strftime('%d/%m/%Y')}",
    height=900,
    xaxis4_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5),
    template="plotly_dark"
)

# Les axes Y pour chaque sous-graphique
fig.update_yaxes(title_text="Prix (USDT)", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)
fig.update_yaxes(title_text="RSI", row=3, col=1, range=[0, 100])
fig.update_yaxes(title_text="MACD", row=4, col=1)

fig.show()

# Résumé de l'analyse technique
print("\n===== Résumé de l'Analyse Technique ETH/USDT =====\n")
print(f"Prix actuel: ${df_recent['close'].iloc[-1]:.2f}")
print(f"Tendance: {trend}")

# Afficher les informations sur les lignes de tendance dynamiques
if trend_supports:
    print("\nLignes de support dynamiques:")
    for i, (importance, a, b, touch_points) in enumerate(trend_supports):
        print(f"  Support {i+1}: {len(touch_points)} points de contact")
        slope_degrees = np.degrees(np.arctan(a))
        print(f"   - Angle: {slope_degrees:.1f}° {'↗' if a > 0 else '↘'}")

if trend_resistances:
    print("\nLignes de résistance dynamiques:")
    for i, (importance, a, b, touch_points) in enumerate(trend_resistances):
        print(f"  Résistance {i+1}: {len(touch_points)} points de contact")
        slope_degrees = np.degrees(np.arctan(a))
        print(f"   - Angle: {slope_degrees:.1f}° {'↗' if a > 0 else '↘'}")

# Signaux des moyennes mobiles
if df_recent['SMA20'].iloc[-1] > df_recent['SMA50'].iloc[-1]:
    signal_ma = "⬆️ Haussier - SMA20 au-dessus de SMA50"
else:
    signal_ma = "⬇️ Baissier - SMA20 en-dessous de SMA50"

print(f"\nSignal MM: {signal_ma}")

# RSI
rsi_value = df_recent['RSI'].iloc[-1]
if rsi_value > 70:
    rsi_signal = "⬇️ Survente - Risque de correction baissière"
elif rsi_value < 30:
    rsi_signal = "⬆️ Surachat - Potentiel de rebond haussier"
else:
    rsi_signal = "➡️ Neutre"

print(f"RSI ({rsi_value:.1f}): {rsi_signal}")

# MACD
if df_recent['MACD'].iloc[-1] > df_recent['MACD_signal'].iloc[-1]:
    macd_signal = "⬆️ Signal haussier - MACD au-dessus du Signal"
else:
    macd_signal = "⬇️ Signal baissier - MACD en-dessous du Signal"

print(f"MACD: {macd_signal}")

# Bandes de Bollinger
price = df_recent['close'].iloc[-1]
upper = df_recent['upper_band'].iloc[-1]
lower = df_recent['lower_band'].iloc[-1]
middle = df_recent['middle_band'].iloc[-1]

bb_width = (upper - lower) / middle
if price > upper * 0.97:
    bb_signal = "⬇️ Proche limite haute - Potentiel retournement baissier"
elif price < lower * 1.03:
    bb_signal = "⬆️ Proche limite basse - Potentiel retournement haussier"
else:
    bb_signal = "➡️ Dans la zone neutre"    

print(f"Bandes de Bollinger: {bb_signal} (Largeur: {bb_width:.2f})")

# Support/Résistance proches
nearest_support = None
nearest_resistance = None
min_support_diff = float('inf')
min_resistance_diff = float('inf')

for _, price_level in support_levels:
    if price > price_level:
        diff = (price - price_level) / price
        if diff < min_support_diff:
            min_support_diff = diff
            nearest_support = price_level

for _, price_level in resistance_levels:
    if price < price_level:
        diff = (price_level - price) / price
        if diff < min_resistance_diff:
            min_resistance_diff = diff
            nearest_resistance = price_level

if nearest_support:
    print(f"Support le plus proche: ${nearest_support:.2f} (-{min_support_diff*100:.2f}%)")
else:
    print("Pas de support identifié dans la plage de prix actuelle")
    
if nearest_resistance:
    print(f"Résistance la plus proche: ${nearest_resistance:.2f} (+{min_resistance_diff*100:.2f}%)")
else:
    print("Pas de résistance identifiée dans la plage de prix actuelle")

print("\n==== Recommandation ====\n")

# Synthèse des signaux pour une recommandation
bullish_signals = 0
bearish_signals = 0

# Signal de tendance
if "Haussière" in trend:
    bullish_signals += 2 if "forte" in trend else 1
elif "Baissière" in trend:
    bearish_signals += 2 if "forte" in trend else 1
    
# Signal MM
if "Haussier" in signal_ma:
    bullish_signals += 1
else:
    bearish_signals += 1
    
# Signal RSI
if "Surachat" in rsi_signal:
    bullish_signals += 1
elif "Survente" in rsi_signal:
    bearish_signals += 1
    
# Signal MACD
if "haussier" in macd_signal:
    bullish_signals += 1
else:
    bearish_signals += 1
    
# Signal BB
if "haussier" in bb_signal.lower():
    bullish_signals += 1
elif "baissier" in bb_signal.lower():
    bearish_signals += 1

# Signaux tendances dynamiques
if trend_supports and any(a > 0 for _, a, _, _ in trend_supports):
    # Support ascendant = haussier
    bullish_signals += 1
    print("Support ascendant détecté: signal haussier (+1)")
elif trend_supports and all(a < 0 for _, a, _, _ in trend_supports):
    # Support descendant = baissier
    bearish_signals += 1
    print("Support descendant détecté: signal baissier (+1)")

if trend_resistances and any(a > 0 for _, a, _, _ in trend_resistances) and any(len(points) >= 3 for _, _, _, points in trend_resistances):
    # Résistance ascendante avec 3+ points = tendance haussière forte
    bullish_signals += 1
    print("Résistance ascendante forte détectée: signal haussier (+1)")

# Recommandation finale
if bullish_signals > bearish_signals + 2:
    recommendation = "ACHAT FORT"
elif bullish_signals > bearish_signals:
    recommendation = "ACHAT"
elif bearish_signals > bullish_signals + 2:
    recommendation = "VENTE FORTE"
elif bearish_signals > bullish_signals:
    recommendation = "VENTE"
else:
    recommendation = "NEUTRE"

print(f"Signaux haussiers: {bullish_signals}")
print(f"Signaux baissiers: {bearish_signals}")
print(f"\nRecommandation: {recommendation}")

Données ETH/USDT 1h de 2017-08-17 04:00:00 à 2025-05-07 08:00:00
Nombre de périodes: 67557
Prix actuel: $1839.04
Variation 24h: 2.28%

Tendance actuelle: Haussière (forte)
Force de la tendance (R²): 0.84
Lignes de support dynamique trouvées: 3
Lignes de résistance dynamique trouvées: 3



===== Résumé de l'Analyse Technique ETH/USDT =====

Prix actuel: $1839.04
Tendance: Haussière (forte)

Lignes de support dynamiques:
  Support 1: 7 points de contact
   - Angle: -34.4° ↘
  Support 2: 9 points de contact
   - Angle: 59.3° ↗
  Support 3: 6 points de contact
   - Angle: 81.2° ↗

Lignes de résistance dynamiques:
  Résistance 1: 4 points de contact
   - Angle: -11.3° ↘
  Résistance 2: 6 points de contact
   - Angle: 80.5° ↗
  Résistance 3: 5 points de contact
   - Angle: -53.9° ↘

Signal MM: ⬇️ Baissier - SMA20 en-dessous de SMA50
RSI (64.5): ➡️ Neutre
MACD: ⬆️ Signal haussier - MACD au-dessus du Signal
Bandes de Bollinger: ⬇️ Proche limite haute - Potentiel retournement baissier (Largeur: 0.07)
Support le plus proche: $1799.70 (-2.14%)
Pas de résistance identifiée dans la plage de prix actuelle

==== Recommandation ====

Support ascendant détecté: signal haussier (+1)
Résistance ascendante forte détectée: signal haussier (+1)
Signaux haussiers: 5
Signaux baissiers: 2

Rec